[@tsatie](https://x.com/tsatie) さんのあみだくじのコードを高速化してみました。

https://github.com/tsatie/SatieGitHubTest/blob/master/20250426AmidaCount.ipynb

あまり良くは理解していませんが，ChatGPT 4o-mini-highを用いました。

ポイントもまとめてもらいました。


以下が高速化のキモとなるポイントです。

1.	全列挙→DP（動的計画法）への置き換え
	- 元コードは長さ L=2a+n のビット列を全パターン 2^L 生成していたが、DP では「現在の高さ」と「現在の並び（perm）」の状態ごとに通り数をまとめて管理。
	- 各ステップで状態数分の遷移を行うだけなので、実質的には O(L \times \text{\#状態}) の計算量に抑えられる。
2.	状態の要約： (高さ, perm)
	- 高さ（現在の累積和）と、縦線の並び替え結果を表す perm（長さ n のベクトル）をキーにしてカウントを保持。
	- 同じ状態が重複して現れたら、その都度再計算せずにカウントを足し合わせることで枝刈りと重複除去を同時に実現。
3.	遷移の単純化
	- 「上へ（+1）」と「下へ（−1）＋スワップ」の２種類だけを考慮。
	- 各状態から最大２つの遷移しか呼び出さないため、１ステップあたりの計算が極めて軽量。
4.	最終ステップだけ特別扱い
	- 最後の１ステップは「高さが n-1 → +1 して n になるもの」を一括で集計。
	- 余分な状態を生成せず、集計ループも一回で済む。
5.	オンラインでの統計更新
	- 最終状態ごとにそのまま stat[i, perm[i]+1] を累積。
	- 別途、全パターンを列挙してから置換を適用する必要がなく、メモリ／時間の両面で効率的。

⸻

これらの工夫により、元の 2^{28} 程度の全探索が瞬時に終了するようになりました。


＜以下は(@tsatie)[https://x.com/tsatie] さんのコード作成の説明です＞

# 標準的な「あみだくじ」でどの筋を選ぶとどの筋に至るかを数える
### Python で組んだものを Claude にお願いして Julia に書き換えて貰った
数え方の要領は, 「あみだくじ」は「行きつ戻りつ」と1:1に対応する（らしい）事を利用している。

例えば, 筋は5本で横線が4本とすれば, $5+4\times2−1$ 回の移動で, 筋から隣の筋への移動をして, 左端から右端まで動くことを「行きつ戻りつ」と表現しよう。
筋を左から 0,1,2,3,4 とでもすれば, 例えば 0→1→0→1→0→1→0→1→0→1→2→3→4 のように5つの筋を「行きつ戻りつ」して最終的に到達するということ。
確かに「戻る」動き(この例では1→0しかない)が4回あることが確認できる。そしてこの「戻る」動きがあみだくじの置換に対応するというアイデアなのだ。

プログラムの概要は以下の通り。筋の数は tn 横棒の数は an としている。
+ 「行きつ戻りつ」を探るために2進数を総当たりして, 行きつ戻りつで辿り着くものを拾う
  - 先ず2進数を順に当たる
  - 文字列化して 0,1 の配列に直し 各項を2倍して1引くことで, -1,1の配列にする
  - その上で配列の先頭からの部分和が 0以上で tn 以下であるもので, 全体和が tn となる（つまり最後では端に到達する）ものを拾う
+ 拾えたらその「行きつ戻りつ」でどの順に筋を経由するかのリストを作成してそこから「あみだくじ」の置換を拾う
  - 「行きつ戻りつ」の行程で「戻る」ときにその2筋を交換していることになる
  - 筋の経由リストから「戻る」部分をピックアップして, 筋の置換を列挙した配列を作る
  - できた置換の配列で順に置換することで, 選んだ筋と結果到達する筋のリストを作成する
+ 「選んだ筋」と「到達する筋」の2次元配列に該当するものを足していく
+ 最終的に分布が得られる。

In [1]:
function calculate_amidakuji_distribution(tn, an)
    # tn: number of vertical lines, an: number of horizontal bars
    ln = 2*an + tn  # total steps

    # Initialize DP: map (height, perm) to count of ways
    dp = Dict{Tuple{Int, Vector{Int}}, Int}()
    dp[(0, collect(0:tn-1))] = 1

    # Iterate through the first ln-1 steps
    for step in 1:ln-1
        new_dp = Dict{Tuple{Int, Vector{Int}}, Int}()
        for ((h, perm), c) in dp
            # Up step (+1)
            if h + 1 <= tn - 1
                key_up = (h + 1, perm)
                new_dp[key_up] = get(new_dp, key_up, 0) + c
            end
            # Down step (-1) and swap if h >= 1
            if h >= 1
                newh = h - 1
                perm2 = copy(perm)
                # Swap columns h and h+1 (1-indexed swap of perm entries)
                perm2[h], perm2[h+1] = perm2[h+1], perm2[h]
                key_dn = (newh, perm2)
                new_dp[key_dn] = get(new_dp, key_dn, 0) + c
            end
        end
        dp = new_dp
    end

    # Final step: only +1 moves that end at height == tn
    stat = zeros(Int, tn, tn)
    total = 0
    for ((h, perm), c) in dp
        if h == tn - 1
            # Apply final +1 => final height = tn
            total += c
            for i in 1:tn
                stat[i, perm[i]+1] += c
            end
        end
    end

    # Output
    println("--start---------------------------------------")
    println("$tn $an")
    println("--Ans-------------------------------------------")
    println(total)
    for i in 1:tn
        println(stat[i, :])
    end
    println("--end-------------------------------------------")
end


calculate_amidakuji_distribution (generic function with 1 method)

In [2]:
calculate_amidakuji_distribution(5, 8)

--start---------------------------------------
5 8
--Ans-------------------------------------------
9841
[4322, 2422, 1627, 1009, 461]
[2422, 2506, 2171, 1733, 1009]
[1627, 2171, 2245, 2171, 1627]
[1009, 1733, 2171, 2506, 2422]
[461, 1009, 1627, 2422, 4322]
--end-------------------------------------------


In [24]:
calculate_amidakuji_distribution(8, 10)

--start---------------------------------------
8 10
--Ans-------------------------------------------
1282735
[764877, 279584, 133631, 64604, 27257, 9481, 2693, 608]
[279584, 478114, 262307, 147260, 72988, 29809, 9980, 2693]
[133631, 262307, 365985, 252938, 153368, 75216, 29809, 9481]
[64604, 147260, 252938, 314118, 250202, 153368, 72988, 27257]
[27257, 72988, 153368, 250202, 314118, 252938, 147260, 64604]
[9481, 29809, 75216, 153368, 252938, 365985, 262307, 133631]
[2693, 9980, 29809, 72988, 147260, 262307, 478114, 279584]
[608, 2693, 9481, 27257, 64604, 133631, 279584, 764877]
--end-------------------------------------------


In [25]:
calculate_amidakuji_distribution(8,12)

--start---------------------------------------
8 12
--Ans-------------------------------------------
16131656
[9188341, 3508269, 1778834, 939616, 451633, 184261, 63000, 17702]
[3508269, 5568480, 3238722, 1961381, 1086206, 507952, 197646, 63000]
[1778834, 3238722, 4168532, 3074842, 2048283, 1130230, 507952, 184261]
[939616, 1961381, 3074842, 3550353, 3019342, 2048283, 1086206, 451633]
[451633, 1086206, 2048283, 3019342, 3550353, 3074842, 1961381, 939616]
[184261, 507952, 1130230, 2048283, 3074842, 4168532, 3238722, 1778834]
[63000, 197646, 507952, 1086206, 1961381, 3238722, 5568480, 3508269]
[17702, 63000, 184261, 451633, 939616, 1778834, 3508269, 9188341]
--end-------------------------------------------


In [26]:
calculate_amidakuji_distribution(8,20)

--start---------------------------------------
8 20
--Ans-------------------------------------------
393151913464
[189723516102, 83314065040, 49329548957, 31599142379, 19875091516, 11372795389, 5626371851, 2311382230]
[83314065040, 105606954285, 72716790819, 52653359557, 37020754687, 23496158993, 12717458232, 5626371851]
[49329548957, 72716790819, 77637484780, 66208709033, 53600482213, 38789943280, 23496158993, 11372795389]
[31599142379, 52653359557, 66208709033, 68259312069, 63935062010, 53600482213, 37020754687, 19875091516]
[19875091516, 37020754687, 53600482213, 63935062010, 68259312069, 66208709033, 52653359557, 31599142379]
[11372795389, 23496158993, 38789943280, 53600482213, 66208709033, 77637484780, 72716790819, 49329548957]
[5626371851, 12717458232, 23496158993, 37020754687, 52653359557, 72716790819, 105606954285, 83314065040]
[2311382230, 5626371851, 11372795389, 19875091516, 31599142379, 49329548957, 83314065040, 189723516102]
--end-------------------------------------------

In [4]:
function amidakuji_count_dp(n::Int, k::Int)
    steps = n + 2k - 1
    max_pos = n - 1

    # dp[step][pos]
    dp = zeros(BigInt, steps + 1, n)

    # 初期状態
    dp[0+1, 0+1] = 1  # (step=0, pos=0)

    for step in 1:steps
        for pos in 0:max_pos
            current = dp[step-1+1, pos+1]

            if pos > 0
                dp[step+1, pos-1+1] += current  # 左に移動
            end
            if pos < max_pos
                dp[step+1, pos+1+1] += current  # 右に移動
            end
        end
    end

    return dp[steps+1, max_pos+1]
end

# 例
println(amidakuji_count_dp(5, 8))  # -> 正しい値が出る

9841


In [6]:
import Pkg; Pkg.add("DataFrames")

┌ Warning: could not download https://pkg.julialang.org/registries
│   exception = Downloads.RequestError("https://pkg.julialang.org/registries", 7, "Failed to connect to jp.pkg.julialang.org port 443 after 210 ms: Couldn't connect to server", Downloads.Response("https", "https://jp.pkg.julialang.org/registries", 301, "HTTP/2 301", ["server" => "Varnish", "retry-after" => "0", "location" => "https://jp.pkg.julialang.org/registries", "x-geo-continent" => "AS", "x-geo-country" => "JP", "x-geo-region" => "13", "accept-ranges" => "bytes", "date" => "Tue, 29 Apr 2025 02:22:33 GMT", "via" => "1.1 varnish", "x-served-by" => "cache-tyo11920-TYO", "x-cache" => "HIT", "x-cache-hits" => "0", "x-timer" => "S1745893353.407184,VS0,VE0", "content-length" => "0"]))
└ @ Pkg.Registry /Users/shimizudan/.julia/juliaup/julia-1.11.5+0.aarch64.apple.darwin14/share/julia/stdlib/v1.11/Pkg/src/Registry/Registry.jl:77
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  

In [7]:
using DataFrames

# 先ほど作ったamidakuji_count_dpをもう一度定義しておく
function amidakuji_count_dp(n::Int, k::Int)
    steps = n + 2k - 1
    max_pos = n - 1

    dp = zeros(BigInt, steps + 1, n)
    dp[1, 1] = 1  # (step=0, pos=0)

    for step in 1:steps
        for pos in 0:max_pos
            current = dp[step, pos+1]
            if pos > 0
                dp[step+1, pos-1+1] += current
            end
            if pos < max_pos
                dp[step+1, pos+1+1] += current
            end
        end
    end

    return dp[steps+1, max_pos+1]
end

# 表を作る
k_range = 1:20
n_range = 3:8

# 中身をためる
table = DataFrame(k = k_range)

for n in n_range
    values = [amidakuji_count_dp(n, k) for k in k_range]
    table[!, "n=$n"] = values
end

# 表示
println(table)

20×7 DataFrame
 Row │ k      n=3      n=4        n=5         n=6          n=7           n=8          
     │ Int64  BigInt   BigInt     BigInt      BigInt       BigInt        BigInt       
─────┼────────────────────────────────────────────────────────────────────────────────
   1 │     1        2          3           4            5             6             7
   2 │     2        4          8          13           19            26            34
   3 │     3        8         21          40           66           100           143
   4 │     4       16         55         121          221           364           560
   5 │     5       32        144         364          728          1288          2108
   6 │     6       64        377        1093         2380          4488          7752
   7 │     7      128        987        3280         7753         15504         28101
   8 │     8      256       2584        9841        25213         53296        100947
   9 │     9      512       6765    

In [1]:
using PrettyTables

function calculate_amidakuji_distribution(tn::Int, an::Int)
    ln = 2*an + tn

    dp = Dict{Tuple{Int, Vector{Int}}, BigInt}()
    dp[(0, collect(0:tn-1))] = 1

    for step in 1:ln-1
        new_dp = Dict{Tuple{Int, Vector{Int}}, BigInt}()
        for ((h, perm), c) in dp
            if h + 1 <= tn - 1
                key_up = (h + 1, perm)
                new_dp[key_up] = get(new_dp, key_up, 0) + c
            end
            if h >= 1
                newh = h - 1
                perm2 = copy(perm)
                perm2[h], perm2[h+1] = perm2[h+1], perm2[h]
                key_dn = (newh, perm2)
                new_dp[key_dn] = get(new_dp, key_dn, 0) + c
            end
        end
        dp = new_dp
    end

    stat = zeros(BigInt, tn, tn)
    total = 0

    for ((h, perm), c) in dp
        if h == tn - 1
            total += c
            for i in 1:tn
                stat[i, perm[i]+1] += c
            end
        end
    end

    println("\n🧮 あみだくじ統計 (n = $tn, k = $an)")
    println("=============================================")
    println("🔢 総パターン数: $total\n")

    println("📊 ゴール別マッピング (回数)")
    pretty_table(stat; 
        header=["Start $(i+1)" for i in 0:tn-1],
        row_labels=["Goal $(i+1)" for i in 0:tn-1],
        compact_printing=false
    )

    println("\n📈 ゴール別マッピング (確率 %)")
    percents = 100 .* stat ./ total
    percents_float = Float64.(percents)
    pretty_table(round.(percents_float; digits=3);
        header=["Start $(i+1)" for i in 0:tn-1],
        row_labels=["Goal $(i+1)" for i in 0:tn-1],
        compact_printing=false
    )

    println("=============================================\n")
end

calculate_amidakuji_distribution (generic function with 1 method)

In [3]:
calculate_amidakuji_distribution(8,10)


🧮 あみだくじ統計 (n = 8, k = 10)
🔢 総パターン数: 1282735

📊 ゴール別マッピング (回数)
┌────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┐
│        │ Start 1 │ Start 2 │ Start 3 │ Start 4 │ Start 5 │ Start 6 │ Start 7 │ Start 8 │
├────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┤
│ Goal 1 │  764877 │  279584 │  133631 │   64604 │   27257 │    9481 │    2693 │     608 │
│ Goal 2 │  279584 │  478114 │  262307 │  147260 │   72988 │   29809 │    9980 │    2693 │
│ Goal 3 │  133631 │  262307 │  365985 │  252938 │  153368 │   75216 │   29809 │    9481 │
│ Goal 4 │   64604 │  147260 │  252938 │  314118 │  250202 │  153368 │   72988 │   27257 │
│ Goal 5 │   27257 │   72988 │  153368 │  250202 │  314118 │  252938 │  147260 │   64604 │
│ Goal 6 │    9481 │   29809 │   75216 │  153368 │  252938 │  365985 │  262307 │  133631 │
│ Goal 7 │    2693 │    9980 │   29809 │   72988 │  147260 │  262307 │  478114 │  279584 │
│ Goal 8 │     608 │    269

In [36]:

function calculate_amidakuji_distribution2(tn::Int, an::Int)
    ln = 2*an + tn

    dp = Dict{Tuple{Int, Vector{Int}}, BigInt}()
    dp[(0, collect(0:tn-1))] = 1

    for step in 1:ln-1
        new_dp = Dict{Tuple{Int, Vector{Int}}, BigInt}()
        for ((h, perm), c) in dp
            if h + 1 <= tn - 1
                key_up = (h + 1, perm)
                new_dp[key_up] = get(new_dp, key_up, 0) + c
            end
            if h >= 1
                newh = h - 1
                perm2 = copy(perm)
                perm2[h], perm2[h+1] = perm2[h+1], perm2[h]
                key_dn = (newh, perm2)
                new_dp[key_dn] = get(new_dp, key_dn, 0) + c
            end
        end
        dp = new_dp
    end

    stat = zeros(BigInt, tn, tn)
    total = 0

    for ((h, perm), c) in dp
        if h == tn - 1
            total += c
            for i in 1:tn
                stat[i, perm[i]+1] += c
            end
        end
    end

    println("\n🧮 あみだくじ統計 (n = $tn, k = $an)")
    println("=============================================")
    println("🔢 総パターン数: $total\n")

    # println("📊 ゴール別マッピング (回数)")
    # pretty_table(stat; 
    #     header=["Start $(i+1)" for i in 0:tn-1],
    #     row_labels=["Goal $(i+1)" for i in 0:tn-1],
    #     compact_printing=false
    # )

    println("\n📈 ゴール別マッピング (確率 %)")
    percents = 100 .* stat ./ total
    percents_float = Float64.(percents)
    pretty_table(round.(percents_float; digits=3);
        header=["Start $(i+1)" for i in 0:tn-1],
        row_labels=["Goal $(i+1)" for i in 0:tn-1],
        compact_printing=false
    )

    println("=============================================\n")
end

calculate_amidakuji_distribution2 (generic function with 1 method)

In [37]:
calculate_amidakuji_distribution2(10,12)


🧮 あみだくじ統計 (n = 10, k = 12)
🔢 総パターン数: 60304951


📈 ゴール別マッピング (確率 %)
┌─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬──────────┐
│         │ Start 1 │ Start 2 │ Start 3 │ Start 4 │ Start 5 │ Start 6 │ Start 7 │ Start 8 │ Start 9 │ Start 10 │
├─────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┼──────────┤
│  Goal 1 │  63.053 │   20.98 │   9.179 │   4.166 │   1.735 │   0.627 │   0.194 │   0.051 │   0.011 │    0.002 │
│  Goal 2 │   20.98 │  41.528 │  19.881 │  10.029 │   4.708 │   1.926 │   0.678 │   0.205 │   0.053 │    0.011 │
│  Goal 3 │   9.179 │  19.881 │  33.375 │  19.275 │  10.443 │   4.918 │   1.983 │   0.689 │   0.205 │    0.051 │
│  Goal 4 │   4.166 │  10.029 │  19.275 │  29.081 │  19.015 │   10.61 │    4.97 │   1.983 │   0.678 │    0.194 │
│  Goal 5 │   1.735 │   4.708 │  10.443 │  19.015 │  27.067 │  18.951 │   10.61 │   4.918 │   1.926 │    0.627 │
│  Goal 6 │   0.627 │   1.92